<a href="https://colab.research.google.com/github/sagacemente/PAN_2023_crypto/blob/main/Crypto_influencers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
!pip install simpletransformers

In [25]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
import os
import pandas as pd
import json
import sklearn
from sklearn.preprocessing import OneHotEncoder
from urllib import request
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [2]:
# Import class Vectorizer
module_url = f"https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/vectorizer.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from vectorizer import Vectorizer

# Import class Simulator
module_url = f"https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/simulator.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))
from simulator import Simulator

Fetching https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/vectorizer.py
Fetching https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/simulator.py
Fetching https://raw.githubusercontent.com/marco-siino/DA-BT/main/code/vectorizer.py


# DS class

In [3]:
class Dataset:

    def __init__(self, url:str, n_subtask:int):
        # Values for ds_name are: "fns", "hss", "iss"
        # Values for language are: "it", "de", "it_de", "ja", "tr", "mix"
        self.url = url
        self.subtask = 'subtask' + str(n_subtask)
        
    def fetch_ds_files(self):
        train_set_archive = tf.keras.utils.get_file('pan23-profiling-cryptocurrency-influencers.zip',self.url,
                                            extract=True, archive_format='zip',cache_dir='.',
                                            cache_subdir='')

    def organize_ds_folders(self):
        train_truth_file_path = os.getcwd() + '/pan23-profiling-cryptocurrency-influencers/' + self.subtask + '/train_truth.json'
        f = open(train_truth_file_path, "r")
        self.id_label_dict = {}
        labels = []
        for line in f:
            line = json.loads(line)
            label = line['class']
            user_id = line['twitter user id']
            self.id_label_dict[user_id] = label
        #print('id_label_dict',self.id_label_dict)

        train_texts_path = os.getcwd() + '/pan23-profiling-cryptocurrency-influencers/'  + self.subtask  + '/train_text.json'
        f = open(train_texts_path, "r")
        self.id_texts_dict = {}
        for line in f:
            line = json.loads(line)
            
            texts = line['texts']
            texts = [i['text'] for i in texts]
            texts = '<NEWTW>'.join(texts)
            user_id = line['twitter user id']
            self.id_texts_dict[user_id] = texts
        #print('id_texts_dict', self.id_texts_dict)

        self.df_texts = pd.DataFrame.from_dict(self.id_texts_dict, orient='index', columns=['text'])
        self.df_labels = pd.DataFrame.from_dict(self.id_label_dict, orient='index', columns=['label'])
        # X = self.df_labels['label'].values.reshape(-1, 1)
        # enc = OneHotEncoder().fit(X)
        # X = enc.transform(X).toarray() #.reshape(-1,5)
        # self.df_labels['label'] = X 
        self.df_labels['label'] = pd.Categorical(self.df_labels['label']).codes

        #Dataframe texts and label
        self.df = pd.concat([self.df_texts, self.df_labels], axis=1)
    
    def generate_keras_ds(self, batch_size, left_size=0.8):
        self.ds = tf.data.Dataset.from_tensor_slices((self.df_texts, self.df_labels))
        self.train_set, self.test_set = tf.keras.utils.split_dataset(self.ds, left_size=left_size)
        # for row in self.train_set.take(3):
        #   print(row)
        # for row in self.test_set.take(3):
        #   print(row)
        self.train_set = self.train_set.shuffle(len(self.train_set),seed=1, reshuffle_each_iteration=False)
        self.test_set =  self.test_set.shuffle(len(self.test_set),seed=1, reshuffle_each_iteration=False)

    def build_ds(self,batch_size, left_size=0.8):
      self.fetch_ds_files()
      self.organize_ds_folders()
      self.generate_keras_ds(batch_size, left_size)

In [4]:
url = 'https://github.com/sagacemente/PAN_2023_crypto/raw/main/pan23-profiling-cryptocurrency-influencers.zip'
ds = Dataset(url, n_subtask= 1)
ds.build_ds(1)

189131/189131 [==============================] - 0s 0us/step


In [ ]:
vct_layer_obj = Vectorizer(ds.train_set)
max_features=len(vct_layer_obj.vectorize_layer.get_vocabulary()) + 1

Length of the longest sample is: 378

Vocabulary size is: 6801


# Models

## CNN

In [ ]:
nr_runs = 2
embedding_dim = 100
nr_epochs = 4

METRICS = [
  tf.keras.metrics.CategoricalAccuracy(name='acc')
  ]

for run in range(1,(nr_runs+1)):
  epochs_accuracy = []
  model = tf.keras.Sequential([
                                  tf.keras.Input(shape=(1,), dtype=tf.string),
                                  vct_layer_obj.vectorize_layer,
                                  layers.Embedding(max_features + 1, embedding_dim),                     
                                  layers.Dropout(0.8),

                                  layers.Conv1D(256,16,activation='relu'),
                                  layers.MaxPooling1D(),
                                  layers.Dropout(0.6),

                                  layers.Dense(512,activation='relu'),
                        
                                  layers.GlobalAveragePooling1D(),
                                  layers.Dropout(0.2),
                                  layers.Dense(5)                            
  ])
  model.compile(loss='sparse_categorical_crossentropy', optimizer='RMSprop', metrics=METRICS) 

  for epoch in range (0,nr_epochs):
      history = model.fit(
        ds.train_set,
        validation_data = ds.test_set,
        epochs=1,
        shuffle=False,
        verbose=1
        # Comment the following line to do not save and download the model.
        #callbacks=[callbacks]
        )
      #accuracy = history.history['val_binary_accuracy']
      #print("Run: ",run,"/ Accuracy on test set at epoch ",epoch," is: ", accuracy[0],"\n")
      #epochs_accuracy.append(accuracy[0])

  #print("Accuracies over epochs:",epochs_accuracy,"\n")

InvalidArgumentError: ignored

## Simple Transformers

In [5]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [35]:
train_df = ds.df[:-10]
train_df.columns = ["text", "labels"]

eval_df = ds.df[-10:]
eval_df.columns = ["text", "labels"]

# Optional model configuration
model_args = ClassificationArgs(num_train_epochs=2, 
                                overwrite_output_dir=True,
                                manual_seed = 4,
                                use_multiprocessing = True,
                                train_batch_size = 32,
                                eval_batch_size = 1)


model_args.custom_parameter_groups = [
    {
        "params": ["acc.sklearn.metrics.f1_score"],
        "average": 'macro',
    },
]

# Create a ClassificationModel
model = ClassificationModel(
    'bert',
    'bert-base-cased',
    num_labels=5,
    args=model_args,
    use_cuda=True,
    ) 

metric = 

# Train the model
model.train_model(train_df,
                  show_running_loss=True,
                  acc=sklearn.metrics.precision_recall_fscore_support
                  )

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df,
                                                            acc=sklearn.metrics.precision_recall_fscore_support)
print('result\n', result)
# Make predictions with the model
#predictions, raw_outputs = model.predict(["Sam was a Wizard"])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/150 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Running Epoch 0 of 2:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 1 of 2:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

result
 {'mcc': -0.04745789978762495, 'acc': (array([0.16666667, 0.        , 0.        , 0.        , 0.        ]), array([1., 0., 0., 0., 0.]), array([0.28571429, 0.        , 0.        , 0.        , 0.        ]), array([1, 2, 1, 4, 2])), 'eval_loss': 1.58740234375}


/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Old code

In [ ]:
# nsubtask = '1'
# subtask = 'subtask' + nsubtask
# subtask

# train_set_archive = tf.keras.utils.get_file('pan23-profiling-cryptocurrency-influencers.zip',url,
#                                     extract=True, archive_format='zip',cache_dir='.',
#                                     cache_subdir='')

# train_truth_file_path = os.getcwd() + '/pan23-profiling-cryptocurrency-influencers/' + subtask + '/train_truth.json'
# f = open(train_truth_file_path, "r")
# id_label_dict = {}
# labels = []
# for line in f:
#     line = json.loads(line)
#     label = line['class']
#     user_id = line['twitter user id']
#     id_label_dict[user_id] = label
# print('id_label_dict',id_label_dict)

# train_texts_path = os.getcwd() + '/pan23-profiling-cryptocurrency-influencers/'  + subtask  + '/train_text.json'
# f = open(train_texts_path, "r")
# id_texts_dict = {}
# for line in f:
#     line = json.loads(line)
    
#     texts = line['texts']
#     texts = [i['text'] for i in texts]
#     texts = '<NEWTW>'.join(texts)
#     user_id = line['twitter user id']
    
#     id_texts_dict[user_id] = texts
# print('id_texts_dict', id_texts_dict)

# df_texts = pd.DataFrame.from_dict(id_texts_dict, orient='index', columns=['text'])
# df_labels = pd.DataFrame.from_dict(id_label_dict, orient='index', columns=['label'])
# df_labels['label'] = pd.Categorical(df_labels['label']).codes
# X = df_labels['label'].values.reshape(-1, 1)
# enc = OneHotEncoder().fit(X)
# X = enc.transform(X).toarray()
# df_labels['label'] = X 
# df = pd.concat([df_texts, df_labels], axis=1)
# ds = tf.data.Dataset.from_tensor_slices((df_texts, df_labels))
# train_ds, test_ds = tf.keras.utils.split_dataset(ds, left_size=0.8)
# for row in train_ds.take(3):
#   print(row)
# for row in test_ds.take(3):
#   print(row)